In [4]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
from datetime import datetime
import random

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

def get_request_with_retries(url, headers, retries=3, timeout=20):
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)
            if response.status_code == 200:
                return response
            else:
                print(f"⚠️ Status {response.status_code} en {url}")
        except Exception as e:
            print(f"⚠️ Intento {i+1} fallido para {url}: {e}")
        time.sleep(2)  # Espera antes de reintentar
    return None

In [ ]:
# Lista de secciones
secciones = ['ocio', 'viajes', 'shopping', 'educacion', 'salud', 'estilo-de-vida']
ciudades = ['madrid', 'barcelona', 'valencia', 'malaga', 'sevilla', 'zaragoza']

articulos = []

for ciudad in ciudades:
    for seccion in secciones:
        print(f"\n🔎 Visitando ciudad: {ciudad.upper()} - sección: {seccion.upper()}")

        for pagina in range(1, 6):  # Hasta 5 páginas por ciudad/sección
            if pagina == 1:
                url = f'https://quehacerconlosninos.es/{ciudad}/{seccion}/'
            else:
                url = f'https://quehacerconlosninos.es/{ciudad}/{seccion}/page/{pagina}/'

            try:
                response = requests.get(url, headers=headers, timeout=30)

                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    titulos = soup.find_all('h3', class_='elementor-post__title')

                    # 🛑 Si no hay artículos, detenemos el bucle de páginas
                    if not titulos:
                        print(f"⚠️ No hay más artículos en {ciudad}/{seccion} página {pagina}. Deteniendo paginado.")
                        break

                    for t in titulos:
                        titulo = t.get_text(strip=True)
                        link = t.find('a')['href']

                        try:
                            # Entrar al artículo con reintentos
                            response_art = get_request_with_retries(link, headers)
                            if response_art:
                                soup_art = BeautifulSoup(response_art.text, 'html.parser')
                                article = soup_art.find('div', class_='elementor-widget-theme-post-content')

                                if article:
                                    parrafos = article.find_all('p')
                                    texto = " ".join(p.get_text(strip=True) for p in parrafos)

                                    if texto.strip():
                                        articulos.append({
                                            'ciudad': ciudad,
                                            'seccion': seccion,
                                            'titulo': titulo,
                                            'url': link,
                                            'contenido': texto
                                        })
                                        print(f"✅ Extraído: {titulo}")
                                    else:
                                        print(f"⚠️ Contenido vacío: {titulo}")
                                else:
                                    print(f"⚠️ No se encontró contenido en: {link}")
                            else:
                                print(f"❌ Fallo en los reintentos para: {link}")

                        except Exception as e:
                            print(f"❌ Error en el artículo {link}: {e}")

                        time.sleep(random.uniform(1.0, 2.5))  # Pausa entre artículos

                else:
                    print(f'❌ No se pudo acceder a {url}')
                    break  # Si falla la página, no seguimos paginando esta sección

            except Exception as e:
                print(f"❌ Error accediendo a {url}: {e}")
                break


In [4]:
df = pd.DataFrame(articulos)

In [5]:
df.to_csv('Articulos3.csv',index=False)

In [5]:
regex_fechas = r"(?:\b(?:lunes|martes|miércoles|jueves|viernes|sábado|domingo)\s*)?\b\d{1,2}\s+de\s+(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)(?:\s+de\s+\d{4})?"
regex_precios = r"\d+(?:[\.,]\d+)? ?€"
meses = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

def extraer_fechas_separadas(texto):
    fechas_encontradas = re.findall(regex_fechas, texto, flags=re.IGNORECASE)

    if not fechas_encontradas:
        return None, None
    elif len(fechas_encontradas) == 1:
        return fechas_encontradas[0], fechas_encontradas[0]
    else:
        return fechas_encontradas[0], fechas_encontradas[1]

def extraer_precio(texto):
    if pd.isnull(texto):
        return "No especificado"
    
    texto = texto.lower()
    
    # Buscar si contiene la palabra "gratis"
    if "gratis" in texto:
        return "Gratis"
    
    # Buscar precios en euros
    precios = re.findall(regex_precios, texto)
    if precios:
        return precios[0].replace(" ", "")
    
    return "No especificado"

def corregir_texto(texto):
    # Insertar espacio entre letras y números
    texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)
    texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
    return texto

def extraer_fecha_contexto_basico(texto):
    if pd.isnull(texto):
        return None

    texto = texto.lower()
    patrones = re.findall(r"(\d{1,2}) de (\w+)(?: de (\d{4}))?", texto)

    if not patrones:
        return None

    for dia, mes_nombre, año in patrones:
        mes = meses.get(mes_nombre)
        if mes is None:
            continue

        año = int(año) if año else datetime.today().year

        try:
            fecha = datetime(int(año), int(mes), int(dia))
            hoy = datetime.today()
            return {
                "fecha_normalizada": fecha.strftime("%Y-%m-%d"),
                "dia_semana": fecha.strftime("%A"),
                "mes": fecha.strftime("%B"),
                "año": fecha.year,
                "es_este_mes": fecha.month == hoy.month and fecha.year == hoy.year,
                "es_este_fin_de_semana": fecha.weekday() in [5, 6] and 0 <= (fecha - hoy).days <= 7
            }
        except ValueError:
            continue

    return None

def extraer_ciudad(texto):
    # Lista de ciudades disponibles
    ciudades = ['Madrid', 'Barcelona', 'Valencia', 'Málaga', 'Sevilla', 'Zaragoza']
    
    patron = r'\b(?:' + '|'.join(ciudades) + r')\b'
    
    # Buscar coincidencias
    coincidencias = re.findall(patron, texto, flags=re.IGNORECASE)
    
    if coincidencias:
        return coincidencias[0].capitalize()
    else:
        return None


In [ ]:
df['contenido'] = df['contenido'].apply(corregir_texto)
df[['fecha_inicio', 'fecha_fin']] = df['contenido'].apply(
    lambda x: pd.Series(extraer_fechas_separadas(x))
)
df['fechas'] = df['contenido'].apply(extraer_fechas_separadas)
df['Precios'] = df['contenido'].apply(extraer_precio)
df['ciudad'] = df['contenido'].apply(extraer_ciudad)
df['fechas_contexto'] = df['contenido'].apply(extraer_fecha_contexto_basico)

In [10]:
df.head()

,seccion,titulo,url,contenido,fecha_inicio,fecha_fin,fechas,Precios,ciudad,fechas_conexto
0,ocio,Motocrossity Oasiz Madrid para niños: adrenali...,https://quehacerconlosninos.es/motocrossity-oa...,Llega a Madrid una propuesta diferente que pro...,None,None,"(None, None)",17€,Madrid,None
1,ocio,Las mejores verbenas de San Isidro 2025: músic...,https://quehacerconlosninos.es/verbenas-san-is...,"Madrid se viste de mantón, clavel y chulapos p...",15 de mayo,15 de mayo,"(15 de mayo, 15 de mayo)",No especificado,Madrid,"{'fecha_normalizada': '2025-05-15', 'dia_seman..."
2,ocio,"San Isidro 2025 en Madrid: tradiciones, verben...",https://quehacerconlosninos.es/san-isidro-2025...,San Isidro 2025 en Madridya tiene fecha: del 3...,15 de mayo,15 de mayo,"(15 de mayo, 15 de mayo)",No especificado,Madrid,"{'fecha_normalizada': '2025-05-15', 'dia_seman..."
3,ocio,Qué hacer en Madrid en mayo con niños: planes ...,https://quehacerconlosninos.es/que-hacer-en-ma...,Mayo es uno de los mejores meses para disfruta...,24 de mayo,30 de mayo,"(24 de mayo, 30 de mayo)",No especificado,Madrid,"{'fecha_normalizada': '2025-05-24', 'dia_seman..."
4,ocio,"Feria del Barro de Aranjuez 2025: talleres, co...",https://quehacerconlosninos.es/feria_del_barro...,Laferia del Barro de Aranjuezregresa este fin ...,domingo 27 de abril,domingo 27 de abril,"(domingo 27 de abril, domingo 27 de abril)",No especificado,Madrid,"{'fecha_normalizada': '2025-04-27', 'dia_seman..."


In [19]:
df.drop(1254,inplace=True)

In [21]:
df['fechas_conexto'].rename({'fechas_conexto':'fechas_contexto'},inplace=True) 

In [22]:
df.to_csv('Articulos_LLM4.csv',index=False)